In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Required Imports
import os

import matplotlib
matplotlib.use('AGG')
import matplotlib.pyplot as plt

import numpy as np

import cv2

from keras.layers import (Input, Activation, Conv3D, Dense, Dropout, Flatten, MaxPooling3D, Input, average, BatchNormalization, LeakyReLU)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import Model
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model

from sklearn.model_selection import train_test_split
from tqdm import tqdm

In [3]:
# Check GPU Availability
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))

#Reduce Precision From float32 To float16
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

#Accelerated Linear Algebra
tf.config.optimizer.set_jit(True)

# Create An Optimizer With Loss Scaling To Prevent Precision Issues
# from tensorflow.keras.mixed_precision import LossScaleOptimizer
# from tensorflow.keras.optimizers import Adam
# opt = LossScaleOptimizer(Adam(learning_rate=1e-4))

Num GPUs Available: 1


In [4]:
#Class To Extract Frames From Video (Converting Video File To 3D Array For Processing)
class Videoto3D:

  def __init__(self, width, height, depth):
    self.width = width
    self.height = height
    self.depth = depth

  #Skip Frames In The Video For Efficiency
  def video3d(self, filename, color=False, skip=True):
    cap = cv2.VideoCapture(filename)
    nframe = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    if skip:
      frames = [x * nframe / self.depth for x in range(self.depth)]
    else:
      frames = [x for x in range(self.depth)]
    framearray = []

    for i in range(self.depth):
      cap.set(cv2.CAP_PROP_POS_FRAMES, frames[i])
      ret, frame = cap.read()
      frame = cv2.resize(frame, (self.height, self.width))
      if color:
        framearray.append(frame)
      else:
        framearray.append(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY))

    cap.release()
    return np.array(framearray) #Outputs A Numpy Array For Each Video

In [5]:
#Function To Load Videos And Labels
def loaddata(video_dir, vid3d, nclass, labellist, result_dir, color=False, skip=True):

  categories = sorted(os.listdir(video_dir)) #One Folder Per Category
  X = []
  labels = []
  labellist = []

  total_categories = len(categories)  # For Progress Bar
  pbar = tqdm(total=total_categories, desc="Processing Directories")

  for category in categories:

    if category in ['.DS_Store', 'output'] :
      continue

    category_path = os.path.join(video_dir, category)

    if category not in labellist:
      if len(labellist) >= nclass: #Only Process Given Amount Of Classes
        break
      labellist.append(category)

    files = os.listdir(category_path)
    for filename in files:

      if filename == '.DS_Store':
        continue

      file_path = os.path.join(category_path, filename)
      X.append(vid3d.video3d(file_path, color=color, skip=skip))
      labels.append(category)

    pbar.update(1)

  pbar.close()

  for num, label in enumerate(labellist): #Assign Numbers For Classes
    for i in range(len(labels)):
      if label == labels[i]:
        labels[i] = num

  if color:
    return np.array(X).transpose((0, 2, 3, 4, 1)), labels #(num_samples, height, width, depth, channels)
  else:
    return np.array(X).transpose((0, 2, 3, 1)), labels #(num_samples, height, width, depth)

In [6]:
#Creating A 3D Convolutional Neural Network With Leaky Relu & Batch Normalization
def create_3dcnn(input_shape, n_classes):
  model = Sequential()

  model.add(Input(shape=input_shape))
  model.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
  model.add(BatchNormalization())  # Standardize activations
  model.add(LeakyReLU(negative_slope=0.01)) # Leaky ReLU instead of ReLU

  model.add(Conv3D(32, kernel_size=(3,3,3), padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(negative_slope=0.01))

  model.add(MaxPooling3D(pool_size=(2, 2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(negative_slope=0.01))

  model.add(Conv3D(64, kernel_size=(3,3,3), padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(negative_slope=0.01))

  model.add(MaxPooling3D(pool_size=(2, 2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv3D(128, kernel_size=(3,3,3), padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(negative_slope=0.01))

  model.add(Conv3D(128, kernel_size=(3,3,3), padding='same'))
  model.add(BatchNormalization())
  model.add(LeakyReLU(negative_slope=0.01))

  model.add(MaxPooling3D(pool_size=(2, 2, 2)))
  model.add(Dropout(0.25))

  model.add(Flatten())
  model.add(Dense(512))
  model.add(BatchNormalization())
  model.add(LeakyReLU(negative_slope=0.01))
  model.add(Dropout(0.5))

  model.add(Dense(n_classes, activation='softmax'))

  return model

In [44]:
#Variables
in_dir = '/content/drive/MyDrive/UCF-101'

out_dir = '/content/drive/MyDrive/UCF-101/output'
if not os.path.isdir(out_dir): #Create Directory If It Doesn't Exist
  os.makedirs(out_dir)

model_weights = '/content/drive/MyDrive/UCF-101/output/model_weights'
if not os.path.isdir(model_weights):
  os.makedirs(model_weights)

results = '/content/drive/MyDrive/UCF-101/output/results'
if not os.path.isdir(results):
  os.makedirs(results)

n_models = 10
n_classes = 101
img_rows,img_cols,frames = 32,32,10

color = False
skip = True

channel = 3 if color else 1

In [8]:
#Saving Each Model's Training/Validation Accuracy & Loss
def plot_history(history, name):
  plt.plot(history.history['accuracy'], marker='.')
  plt.plot(history.history['val_accuracy'], marker='.')
  plt.title('model accuracy')
  plt.xlabel('epoch')
  plt.ylabel('accuracy')
  plt.grid()
  plt.savefig(os.path.join(results, '{}_accuracy.png'.format(name)))
  plt.close()

  plt.plot(history.history['loss'], marker='.')
  plt.plot(history.history['val_loss'], marker='.')
  plt.title('model loss')
  plt.xlabel('epoch')
  plt.ylabel('loss')
  plt.grid()
  plt.savefig(os.path.join(results, '{}_loss.png'.format(name)))
  plt.close()

In [9]:
#Function To Load Training & Testing Data
def Train_Test_Data():

  vid3d = Videoto3D(img_rows, img_cols, frames)

  preloaded_data_path = '/content/drive/MyDrive/UCF-101/output/dataset.npz'

  if os.path.exists(preloaded_data_path): #Load Data If It Already Exists
    loadeddata = np.load(preloaded_data_path)
    X, Y = loadeddata["X"], loadeddata["Y"]
  else:
    x, y = loaddata(in_dir, vid3d, n_classes, out_dir, color, skip) #Load And Save Data If It Doesn't Exist
    X = x.reshape((x.shape[0], img_rows, img_cols, frames, channel))
    Y = to_categorical(y, n_classes)

    X = X.astype('float16')
    np.savez(preloaded_data_path, X=X, Y=Y)
  print('X_shape:{} Y_shape:{}'.format(X.shape, Y.shape))

  return train_test_split(X, Y, test_size=0.2), X.shape[1: ]

(X_train, X_test, Y_train, Y_test), in_shape = Train_Test_Data()

X_shape:(13320, 32, 32, 10, 1) Y_shape:(13320, 101)


In [10]:
#Function To Train An Ensemble Of Models
def train_model(epochs=100, batch_size=128):

  #Train Test Generator
  train_dataset = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(len(X_train)).batch(batch_size).prefetch(tf.data.AUTOTUNE)
  test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(batch_size).prefetch(tf.data.AUTOTUNE)

  #Implementing Model CallBacks
  early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, verbose=1)
  reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6, verbose=1)

  #Training Models Individually
  models=[]
  for i in range(n_models):
    print(f"Model {i}:")
    models.append(create_3dcnn(in_shape, n_classes))
    models[-1].compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    model_path = os.path.join(model_weights, f"model_{i}.h5")

    #Load Weights If Model Already Trained
    if os.path.exists(model_path):
      models[-1] = load_model(model_path)
      print(f"Model {i} weights loaded from: {model_path}")

    #Train The Model If Weights Do Not Exist
    else:
      history = models[-1].fit(train_dataset, validation_data=test_dataset, epochs=epochs, callbacks=[early_stopping, reduce_lr], verbose=1)
      models[-1].save(model_path)
      print(f"Model {i} weights saved at: {model_path}")
      plot_history(history, i)

  #Creating An Ensemble Model
  model_inputs = [Input(shape=in_shape) for _ in range (n_models)]
  model_outputs = [models[i](model_inputs[i]) for i in range (n_models)]
  model_outputs = average(inputs=model_outputs)
  model = Model(inputs=model_inputs, outputs=model_outputs)
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

  model.summary()

  model.save(os.path.join(out_dir, 'ucf101_3dcnnmodel.h5'))

  X_test_copies = [np.copy(X_test) for _ in range(n_models)]
  loss, acc = model.evaluate(X_test_copies, Y_test, verbose=0)
  with open(os.path.join(out_dir, 'result.txt'), 'w') as f:
    f.write(f"Test loss: {loss:.4f}, Test accuracy: {acc:.4f}")

  print('merged model:')
  print('Test loss:', loss)
  print('Test accuracy:', acc)

In [ ]:
train_model() #Default Hyperparameters

Model 0:
Epoch 1/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 53s 264ms/step - accuracy: 0.0309 - loss: 4.9982 - val_accuracy: 0.0762 - val_loss: 4.9609 - learning_rate: 0.0010
Epoch 2/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - accuracy: 0.1341 - loss: 3.7418 - val_accuracy: 0.0360 - val_loss: 5.6272 - learning_rate: 0.0010
Epoch 3/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 58ms/step - accuracy: 0.2456 - loss: 3.0189 - val_accuracy: 0.1580 - val_loss: 4.2203 - learning_rate: 0.0010
Epoch 4/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - accuracy: 0.3559 - loss: 2.5003 - val_accuracy: 0.3607 - val_loss: 2.5204 - learning_rate: 0.0010
Epoch 5/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - accuracy: 0.4451 - loss: 2.1295 - val_accuracy: 0.4133 - val_loss: 2.3021 - learning_rate: 0.0010
Epoch 6/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.5233 - loss: 1.7859 - val_accuracy: 0.4872 - val_loss: 1.9979 - learning_rate: 0.0010
Epoch 7/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.5984 - loss

Model 0 weights saved at: /content/drive/MyDrive/UCF-101/output/model_weights/model_0.h5
Model 1:
Epoch 1/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 29s 176ms/step - accuracy: 0.0218 - loss: 5.0872 - val_accuracy: 0.0518 - val_loss: 6.3325 - learning_rate: 0.0010
Epoch 2/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.1304 - loss: 3.7882 - val_accuracy: 0.1201 - val_loss: 3.9130 - learning_rate: 0.0010
Epoch 3/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.2419 - loss: 3.0220 - val_accuracy: 0.2335 - val_loss: 3.1110 - learning_rate: 0.0010
Epoch 4/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - accuracy: 0.3450 - loss: 2.5713 - val_accuracy: 0.3161 - val_loss: 2.7233 - learning_rate: 0.0010
Epoch 5/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.4444 - loss: 2.1104 - val_accuracy: 0.4471 - val_loss: 2.1480 - learning_rate: 0.0010
Epoch 6/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.5298 - loss: 1.7915 - val_accuracy: 0.5150 - val_loss: 1.8535 - learning_r

Model 1 weights saved at: /content/drive/MyDrive/UCF-101/output/model_weights/model_1.h5
Model 2:
Epoch 1/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 28s 163ms/step - accuracy: 0.0297 - loss: 5.0034 - val_accuracy: 0.0458 - val_loss: 5.5848 - learning_rate: 0.0010
Epoch 2/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 24s 63ms/step - accuracy: 0.1375 - loss: 3.7129 - val_accuracy: 0.2091 - val_loss: 3.1782 - learning_rate: 0.0010
Epoch 3/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 10s 63ms/step - accuracy: 0.2546 - loss: 2.9853 - val_accuracy: 0.2793 - val_loss: 2.8932 - learning_rate: 0.0010
Epoch 4/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.3709 - loss: 2.4476 - val_accuracy: 0.3956 - val_loss: 2.2851 - learning_rate: 0.0010
Epoch 5/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.4476 - loss: 2.1124 - val_accuracy: 0.4167 - val_loss: 2.2658 - learning_rate: 0.0010
Epoch 6/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 61ms/step - accuracy: 0.5404 - loss: 1.7319 - val_accuracy: 0.5169 - val_loss: 1.9074 - learning

Model 2 weights saved at: /content/drive/MyDrive/UCF-101/output/model_weights/model_2.h5
Model 3:
Epoch 1/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 27s 178ms/step - accuracy: 0.0271 - loss: 5.0366 - val_accuracy: 0.0360 - val_loss: 9.6204 - learning_rate: 0.0010
Epoch 2/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - accuracy: 0.1338 - loss: 3.7881 - val_accuracy: 0.1479 - val_loss: 3.6520 - learning_rate: 0.0010
Epoch 3/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.2604 - loss: 2.9829 - val_accuracy: 0.2286 - val_loss: 3.2422 - learning_rate: 0.0010
Epoch 4/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 60ms/step - accuracy: 0.3727 - loss: 2.4489 - val_accuracy: 0.2639 - val_loss: 2.9730 - learning_rate: 0.0010
Epoch 5/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 63ms/step - accuracy: 0.4509 - loss: 2.0857 - val_accuracy: 0.4486 - val_loss: 2.1183 - learning_rate: 0.0010
Epoch 6/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 10s 61ms/step - accuracy: 0.5408 - loss: 1.7418 - val_accuracy: 0.5586 - val_loss: 1.7253 - learning_

Model 3 weights saved at: /content/drive/MyDrive/UCF-101/output/model_weights/model_3.h5
Model 4:
Epoch 1/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 29s 175ms/step - accuracy: 0.0290 - loss: 4.9866 - val_accuracy: 0.0601 - val_loss: 5.8032 - learning_rate: 0.0010
Epoch 2/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 59ms/step - accuracy: 0.1371 - loss: 3.7294 - val_accuracy: 0.2218 - val_loss: 3.1558 - learning_rate: 0.0010
Epoch 3/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 62ms/step - accuracy: 0.2500 - loss: 3.0225 - val_accuracy: 0.1569 - val_loss: 3.7542 - learning_rate: 0.0010
Epoch 4/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 10s 60ms/step - accuracy: 0.3523 - loss: 2.4941 - val_accuracy: 0.3179 - val_loss: 2.8743 - learning_rate: 0.0010
Epoch 5/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 61ms/step - accuracy: 0.4453 - loss: 2.1062 - val_accuracy: 0.3934 - val_loss: 2.3815 - learning_rate: 0.0010
Epoch 6/100
84/84 ━━━━━━━━━━━━━━━━━━━━ 5s 61ms/step - accuracy: 0.5366 - loss: 1.7545 - val_accuracy: 0.5522 - val_loss: 1.7811 - learning_

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_26            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_27            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_28            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_29            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_30            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_31            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_32            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_33            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_34            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_35            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_16             │ (None, 101)            │      1,963,141 │ input_layer_26[0][0]   │
│ (Sequential)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_17             │ (None, 101)            │      1,963,141 │ input_layer_27[0][0]   │
│ (Sequential)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_18             │ (None, 101)            │      1,963,141 │ input_layer_28[0][0]   │
│ (Sequential)              │                        │                │                        │
├──────────────────────

 Total params: 19,631,410 (74.89 MB)

 Trainable params: 19,612,210 (74.81 MB)

 Non-trainable params: 19,200 (75.00 KB)

merged model:
Test loss: 0.5044229626655579
Test accuracy: 0.8802552819252014


In [40]:
#Load Model For Prediction
model_path = os.path.join(out_dir, 'ucf101_3dcnnmodel.h5')
model = load_model(model_path, custom_objects={'average': average})
model.summary()

Model: "functional_1066"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_26            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_27            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_28            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_29            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_30            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_31            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_32            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_33            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_34            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ input_layer_35            │ (None, 32, 32, 10, 1)  │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_16             │ (None, 101)            │      1,963,141 │ input_layer_26[0][0]   │
│ (Sequential)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_17             │ (None, 101)            │      1,963,141 │ input_layer_27[0][0]   │
│ (Sequential)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ sequential_18             │ (None, 101)            │      1,963,141 │ input_layer_28[0][0]   │
│ (Sequential)              │                        │                │                        │
├──────────────────────

 Total params: 19,631,416 (74.89 MB)

 Trainable params: 19,612,210 (74.81 MB)

 Non-trainable params: 19,200 (75.00 KB)

 Optimizer params: 6 (36.00 B)

In [47]:
#Function To Load Category Labels
def load_labels():
  labellist = []
  categories = sorted(os.listdir(in_dir)) #One Folder Per Category

  for category in categories:
    if category in ['.DS_Store', 'output']: #Ensure Only Categories Are Read
      continue

    category_path = os.path.join(in_dir, category)

    if category not in labellist:
      if len(labellist) >= n_classes: #Only Process Given Amount Of Classes
        break
      labellist.append(category)
  return labellist

#Function For Testing Model Predictions On Samples
def test_model_on_example(model, X_test, Y_test, labellist, example_index=0):
  sample = X_test[example_index]
  sample = np.expand_dims(sample, axis=0)
  sample = [np.copy(sample) for _ in range(n_models)]
  true_label = np.argmax(Y_test[example_index])

  prediction = model.predict(sample)
  predicted_label = np.argmax(prediction)

  print(f"True Label: {labellist[true_label]}")
  print(f"Predicted Label: {labellist[predicted_label]}")

  return true_label, predicted_label


labellist = load_labels()

correct_predictions = 0
total_predictions = 0
for i in range(100): #First 100 Sample Predictions
  true_label, predicted_label = test_model_on_example(model, X_test, Y_test, labellist, example_index=i)
  total_predictions += 1
  if true_label == predicted_label:
    correct_predictions += 1

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
True Label: YoYo
Predicted Label: PizzaTossing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
True Label: Basketball
Predicted Label: Basketball
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
True Label: BaseballPitch
Predicted Label: BaseballPitch
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
True Label: Diving
Predicted Label: Diving
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
True Label: Drumming
Predicted Label: Drumming
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
True Label: PlayingCello
Predicted Label: PlayingCello
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
True Label: PlayingSitar
Predicted Label: PlayingSitar
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
True Label: BandMarching
Predicted Label: BandMarching
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
True Label: Bowling
Predicted Label: Bowling
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
True Label: BlowDryHair
Predicted Label: BlowDryHair
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
True Label: PlayingSitar
Predicted Label: PlayingSitar
1/1 ━━━━━━━━━━━━━━━

In [48]:
accuracy = correct_predictions / total_predictions #Calculate Accuracy For 100 Samples In Test Data
print(f"Accuracy: {accuracy * 100:.2f}%")

Accuracy: 96.00%
